In [1]:
import os.path as path
import glob, os
from datetime import datetime
from importlib.metadata import version
import cobra
import thermo_flux
from thermo_flux.io import load_excel as ex
from thermo_flux.core.model import ThermoModel
from equilibrator_api import  Q_
import pandas as pd
from thermo_flux.io import helper_load as hl
import numpy as np
from thermo_flux.io import load_excel as ex
from scripts.logger import write_to_log
import gurobipy as gp
from gurobipy import GRB
from scripts.gen_model import gen_model
from scripts.gen_model import apply_physio_data
from scripts.gen_model import constrain_bounds_fva
from scripts.reaction_utils import list_blocked_reactions
from scripts.reaction_utils import count_blocked_pathways
from scripts.reaction_utils import plot_calc_vs_exp
from thermo_flux.solver.gurobi import compute_IIS
import importlib

In [2]:
INPUT_MODEL = "datafiles/model.xlsx"
INPUT_KEGGS = "datafiles/ecoli_kegg_id.csv"
INPUT_REED = "regression/reed.csv"
INPUT_INCHI = "regression/InChIs.csv"
INPUT_GAMS = "regression/model_Ecoli_from-gams.xlsx"
INPUT_EXP_DATA = "regression/allPhysioData_formatted_forGSM_20230831.csv"
INPUT_EXP_CONC = "regression/allConcRange_20230912.csv"
INPUT_METABOLOMICS = "regression/metabolomics-Kochanowski_20230925.csv"

MODEL_NAME = "ecoli"

OUTPUT_DIR = "output"
OUTPUT_NAME = f"dev_conc"
OUTPUT_LOG = f"{OUTPUT_DIR}{path.sep}{OUTPUT_NAME}_log.txt"

CONDITIONS_TO_REGRESS = ["WT-Glc_I", "WT-Gal_I", "WT-Fruc_I", "WT-Mann_I", "dptsG-Glc_I", 
                         "WT-Ace_I", "WT-Succ_I", "WT-Fum_I", "WT-Glyc_I", "WT-Pyr_I",
                         "WT-GlyCAA_II"]

CONDITIONS_TO_REGRESS = ["WT-Glyc_I"]

INCLUDE_CO2 = True
INCLUDE_O2 = True
ALLOW_OTHER_EXCRETION = False
RELAX_EXP_FLUX_BOUNDS = 2.0

time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
write_to_log(OUTPUT_LOG, f"Started analysis at: {time}", "w")

# Write package versions:
modules = ["pandas", "numpy", "equilibrator_api", "cobra"]
write_to_log(OUTPUT_LOG, f"Package versions used:")
versions_packages = [f"  {m}: {version(m)}\n" for m in modules]
write_to_log(OUTPUT_LOG, "".join(versions_packages))

In [5]:
importlib.reload(thermo_flux.core.model)
importlib.reload(thermo_flux.solver.gurobi)

<module 'thermo_flux.solver.gurobi' from 'C:\\Users\\iLawz\\Documents\\Thermoflux\\thermo-flux-main\\thermo_flux\\solver\\gurobi.py'>

In [8]:
importlib.reload(thermo_flux.solver.gurobi)

<module 'thermo_flux.solver.gurobi' from 'C:\\Users\\iLawz\\Documents\\Thermoflux\\thermo-flux-main\\thermo_flux\\solver\\gurobi.py'>

In [3]:
tmodel = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-02
['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' created
unknown metabolite '3hpppn[c]' created
unknown metabolite 'dhpppn[c]' created
unknown metabolite 'phthr[c]' created
unknown metabolite '4hthr[c]' created
unknown m

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


*** Updating metabolite information ***
2dhglcn_c NOTHING DONE!
nadh_c NOTHING DONE!
glcn_c NOTHING DONE!
nad_c NOTHING DONE!
nadph_c NOTHING DONE!
nadp_c NOTHING DONE!
2dhguln_c NOTHING DONE!
idon-L_c NOTHING DONE!
3hcinnm_c NOTHING DONE!
o2_c NOTHING DONE!
dhcinnm_c NOTHING DONE!
h2o_c NOTHING DONE!
3hpppn_c NOTHING DONE!
dhpppn_c NOTHING DONE!
phthr_c NOTHING DONE!
4hthr_c NOTHING DONE!
pi_c NOTHING DONE!
5dglcn_c NOTHING DONE!
ru5p-D_c NOTHING DONE!
ara5p_c NOTHING DONE!
ACP_c NOTHING DONE!
atp_c NOTHING DONE!
ttdca_c NOTHING DONE!
amp_c NOTHING DONE!
myrsACP_c NOTHING DONE!
ppi_c NOTHING DONE!
ttdcea_c NOTHING DONE!
tdeACP_c NOTHING DONE!
hdca_c NOTHING DONE!
palmACP_c NOTHING DONE!
hdcea_c NOTHING DONE!
hdeACP_c NOTHING DONE!
ocdcea_c NOTHING DONE!
octeACP_c NOTHING DONE!
dtdp4aaddg_c NOTHING DONE!
unagamu_c NOTHING DONE!
dtdp_c NOTHING DONE!
unagamuf_c NOTHING DONE!
arbt6p_c NOTHING DONE!
g6p_c NOTHING DONE!
hqn_c NOTHING DONE!
4abut_c NOTHING DONE!
akg_c NOTHING DONE!
glu-L_c N

In [4]:
print(tmodel.reactions.biomass_ce.drG)
print(tmodel.reactions.biomass_ce.drG0prime)
print(tmodel.reactions.biomass_ce.drGtransform)
print(tmodel.reactions.biomass_ce.drGtransport)

print(tmodel.reactions.biomass_EX.drG)
print(tmodel.reactions.biomass_EX.drG0prime)
print(tmodel.reactions.biomass_EX.drGtransform)
print(tmodel.reactions.biomass_EX.drGtransport)

None
0.0 kilojoule / mole
None
263.4944475888886 kilojoule / mole
None
-709.01190797458 kilojoule / mole
None
0 kilojoule / mole


In [5]:
tmodel = apply_physio_data(tmodel, CONDITIONS_TO_REGRESS[0], INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, ALLOW_OTHER_EXCRETION, OUTPUT_LOG)

['EX_12ppd-S', 'EX_15dap', 'EX_26dap-M', 'EX_2ddglcn', 'EX_3hcinnm', 'EX_3hpppn', 'EX_4abut', 'EX_ac', 'EX_acac', 'EX_acald', 'EX_acgam', 'EX_acmana', 'EX_acnam', 'EX_ade', 'EX_adn', 'EX_akg', 'EX_ala-D', 'EX_ala-L', 'EX_alltn', 'EX_amp', 'EX_arab-L', 'EX_arg-L', 'EX_asn-L', 'EX_asp-L', 'EX_but', 'EX_C', 'EX_cbl1', 'EX_chol', 'EX_cit', 'EX_co2', 'EX_crn', 'EX_csn', 'EX_cynt', 'EX_cys-L', 'EX_cytd', 'EX_dad-2', 'EX_dcyt', 'EX_dgsn', 'EX_dha', 'EX_din', 'EX_dms', 'EX_dmso', 'EX_duri', 'EX_etoh', 'EX_fe2', 'EX_for', 'EX_fru', 'EX_fuc1p-L', 'EX_fuc-L', 'EX_fum', 'EX_g6p', 'EX_gal', 'EX_galct-D', 'EX_galctn-D', 'EX_galt', 'EX_galur', 'EX_gam', 'EX_gbbtn', 'EX_glc', 'EX_glcn', 'EX_glcr', 'EX_glcur', 'EX_gln-L', 'EX_glu-L', 'EX_gly', 'EX_glyald', 'EX_glyb', 'EX_glyc', 'EX_glyc3p', 'EX_glyclt', 'EX_gsn', 'EX_gua', 'EX_h', 'EX_h2o', 'EX_hdca', 'EX_his-L', 'EX_hxan', 'EX_idon-L', 'EX_ile-L', 'EX_indole', 'EX_ins', 'EX_k', 'EX_lac-D', 'EX_lac-L', 'EX_lcts', 'EX_leu-L', 'EX_lys-L', 'EX_mal-L', 'EX

In [3]:
tmodel = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)
tmodel = apply_physio_data(tmodel, "WT-Glc_I", INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, False, OUTPUT_LOG)
blocked_p = list_blocked_reactions(tmodel, "WT-Glc_I", OUTPUT_LOG, 1, False)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-02
['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' created
unknown metabolite '3hpppn[c]' created
unknown metabolite 'dhpppn[c]' created
unknown metabolite 'phthr[c]' created
unknown metabolite '4hthr[c]' created
unknown m

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


*** Updating metabolite information ***
2dhglcn_c NOTHING DONE!
nadh_c NOTHING DONE!
glcn_c NOTHING DONE!
nad_c NOTHING DONE!
nadph_c NOTHING DONE!
nadp_c NOTHING DONE!
2dhguln_c NOTHING DONE!
idon-L_c NOTHING DONE!
3hcinnm_c NOTHING DONE!
o2_c NOTHING DONE!
dhcinnm_c NOTHING DONE!
h2o_c NOTHING DONE!
3hpppn_c NOTHING DONE!
dhpppn_c NOTHING DONE!
phthr_c NOTHING DONE!
4hthr_c NOTHING DONE!
pi_c NOTHING DONE!
5dglcn_c NOTHING DONE!
ru5p-D_c NOTHING DONE!
ara5p_c NOTHING DONE!
ACP_c NOTHING DONE!
atp_c NOTHING DONE!
ttdca_c NOTHING DONE!
amp_c NOTHING DONE!
myrsACP_c NOTHING DONE!
ppi_c NOTHING DONE!
ttdcea_c NOTHING DONE!
tdeACP_c NOTHING DONE!
hdca_c NOTHING DONE!
palmACP_c NOTHING DONE!
hdcea_c NOTHING DONE!
hdeACP_c NOTHING DONE!
ocdcea_c NOTHING DONE!
octeACP_c NOTHING DONE!
dtdp4aaddg_c NOTHING DONE!
unagamu_c NOTHING DONE!
dtdp_c NOTHING DONE!
unagamuf_c NOTHING DONE!
arbt6p_c NOTHING DONE!
g6p_c NOTHING DONE!
hqn_c NOTHING DONE!
4abut_c NOTHING DONE!
akg_c NOTHING DONE!
glu-L_c N

In [4]:
tmodel.remove_reactions(blocked_p, remove_orphans=True)
for rxn in tmodel.reactions:
    thermo_flux.tools.drg_tools.reaction_balance(rxn, balance_charge=True, balance_mg=False)
tmodel.update_thermo_info(fit_unknown_dfG0=True)

Identifying compounds...
[████████████████████████████████████████] 463/463 Mg_e                          

Estimating dfG0'...
[████████████████████████████████████████] 463/463 Mg_e                                                                                                          

Estimating drG0'...
[████████████████████████████████████████] 623/623 EX_oro                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 



In [5]:
tmodel.m = None
tmodel.objective = tmodel.reactions.biomass_EX
tmodel.add_TFBA_variables(lnc_unit="M")

Using conc unit: M
Using scaling term: 1
Using correction vector of [-0. -0. -0. -0.  0. -0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0. -0.
  0. -0. -0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. -0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. -0.  0. -0.  0. -0.  0.  0.  0. -0. -0.  0.  0.
  0.  0. -0.  0.  0. -0. -0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.
  0. -0. -0.  0.  0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.  0. -0. -0.  0. -0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0. -0.  0. -0.  0.  0.  0.  0.
  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0.  0.  0.
  0. -0. -0.  0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0. -0. -0.  0.

In [6]:
tmodel.m.Params.TimeLimit = 10
tmodel.m.optimize()
# MOLAR

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6449 rows, 5132 columns and 158399 nonzeros
Model fingerprint: 0xaef3842e
Model has 1 general constraint
Variable types: 4509 continuous, 623 integer (623 binary)
Coefficient statistics:
  Matrix range     [3e-06, 7e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e-01, 1e+06]
  RHS range        [2e-14, 1e+05]
Presolve removed 3165 rows and 2503 columns
Presolve time: 0.38s
Presolved: 3285 rows, 2935 columns, 140920 nonzeros
Presolved model has 306 quadratic constraint(s)
Variable types: 2431 continuous, 504 integer (504 binary)

Root relaxation: objective 7.018798e-01, 2506 iterations, 0.22 seconds (0.67 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

In [8]:
tmodel.m = None
tmodel.objective = tmodel.reactions.biomass_EX
tmodel.add_TFBA_variables(lnc_unit="mM") 
# MILLI MOLAR

Using conc unit: mM
Using scaling term: 1000.0
Using correction vector of [-1.78036789e+01 -1.78036789e+01 -1.78036789e+01 -1.78036789e+01
  0.00000000e+00 -3.40237922e+02 -3.40237922e+02 -3.40237922e+02
 -3.40237922e+02 -3.40237922e+02  0.00000000e+00  1.78036789e+01
  0.00000000e+00  1.78036789e+01  0.00000000e+00  0.00000000e+00
  1.78036789e+01 -1.78036789e+01  0.00000000e+00 -1.78036789e+01
 -3.56073578e+02  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.58356559e+01 -1.58818964e+01  3.19494113e+01  3.65794645e+00
 -1.38768271e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.78036789e+01  1.58356559e+01  1.78036789e+01  1.78036789e+01
  1.78036789e+01  3.56073578e+01  0.00000000e+00  0.00000000e+00
 -1.78036789e+01  2.16055773e+01  0.00000000e+00  1.78036789e+01
  0.00000000e+00 -8.81531941e-01  0.00000000e+00 -8.81531941e-01
  1.78036789e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.58356559e+01  3.56073578e+01
 -2.07579278e+01

In [ ]:
tmodel.m.Params.TimeLimit = 5
tmodel.m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6449 rows, 5132 columns and 158399 nonzeros
Model fingerprint: 0x46cfdd0a
Model has 1 general constraint
Variable types: 4509 continuous, 623 integer (623 binary)
Coefficient statistics:
  Matrix range     [3e-06, 7e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-02, 1e+06]
  RHS range        [2e-14, 1e+05]
Presolve removed 3165 rows and 2503 columns
Presolve time: 0.38s
Presolved: 3285 rows, 2935 columns, 140920 nonzeros
Presolved model has 306 quadratic constraint(s)
Variable types: 2431 continuous, 504 integer (504 binary)

Root relaxation: objective 7.018798e-01, 2304 iterations, 0.19 seconds (0.53 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O